In [1]:
import pandas as pd
import numpy as np
import sqlite3 as sl

In [2]:
# соединение с базой sqlite
con = sl.connect('my-test.db')

In [3]:
members_13 = pd.read_csv('members_13.csv', parse_dates=['birthday','entry_date'])
members_13

,member_name,birthday,sex,profession,salary_usd$,entry_date,group_id,city_name,state_name,age,fun_club_name,category_id,monthly_fee_USD$,category_name,membership_duration_days,"sum_fee_paid, USD$"
0,Vanessa Casey,1960-08-08,F,"Engineer, manufacturing systems",1884,2018-05-24,19927387,Montgomery,West Virginia,63,Events That Make You Smarter,6,7,Education & Learning,1803,421.0
1,Cheryl Carey,1958-12-11,N,Trade union research officer,1518,2019-11-14,19712846,Colonia,New Jersey,65,Mix Professionals Chicago,9,18,Fitness,1264,758.0
2,Kendra Valdez,1962-06-16,M,Press photographer,1669,2018-08-06,10528562,Pinckney,Michigan,61,Madison Square Park Mommies!,25,19,Parents & Family,1729,1095.0
3,Toni Baxter,1992-03-21,F,Emergency planning/management officer,3818,2017-07-18,1291778,Florence,Kentucky,31,San Francisco Dodgeball Meetup Group,32,1,Sports & Recreation,2113,70.0
4,Shaun Zuniga,1999-07-04,N,"Runner, broadcasting/film/video",4962,2017-11-18,26053931,Nickerson,Kansas,24,Passionate Concert Goers,21,9,Music,1990,597.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130984,George Mcdaniel,1943-03-14,M,Comptroller,2393,2018-04-14,1222759,Oakhurst,California,80,RHYTHMflow Explorations,21,21,Music,1843,1290.0
130985,Kevin Diaz DVM,1999-05-16,F,Customer service manager,4874,2018-12-27,2137821,Spring Lake Park,Minnesota,24,Aquatic Voyagers Scuba Club,23,18,Outdoors & Adventure,1586,952.0
130986,Latasha Alvarez MD,1953-07-07,N,Interpreter,2579,2018-09-20,23343110,Newton County,Missouri,70,Chicago Womens Soccer Meet up,32,5,Sports & Recreation,1684,281.0
130987,Hector Davidson,1945-05-05,N,Lexicographer,2890,2018-11-18,10982362,Houston,Alaska,78,Chicago Indonesian (Bahasa Indonesia) Language...,16,23,Language & Ethnic Identity,1625,1246.0


In [4]:
members_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130989 entries, 0 to 130988
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   member_name               130989 non-null  object        
 1   birthday                  130989 non-null  datetime64[ns]
 2   sex                       130989 non-null  object        
 3   profession                130989 non-null  object        
 4   salary_usd$               130989 non-null  int64         
 5   entry_date                130989 non-null  datetime64[ns]
 6   group_id                  130989 non-null  int64         
 7   city_name                 130989 non-null  object        
 8   state_name                130989 non-null  object        
 9   age                       130989 non-null  int64         
 10  fun_club_name             130974 non-null  object        
 11  category_id               130989 non-null  int64         
 12  mo

In [6]:
# загружаем датасет в базу sqllite
members_13.to_sql('members_13', con)

130989

In [7]:
pd.read_sql('''SELECT* FROM members_13 LIMIT 3''', con, index_col='index')

,member_name,birthday,sex,profession,salary_usd$,entry_date,group_id,city_name,state_name,age,fun_club_name,category_id,monthly_fee_USD$,category_name,membership_duration_days,"sum_fee_paid, USD$"
index,,,,,,,,,,,,,,,,
0,Vanessa Casey,1960-08-08 00:00:00,F,"Engineer, manufacturing systems",1884,2018-05-24 00:00:00,19927387,Montgomery,West Virginia,63,Events That Make You Smarter,6,7,Education & Learning,1803,421.0
1,Cheryl Carey,1958-12-11 00:00:00,N,Trade union research officer,1518,2019-11-14 00:00:00,19712846,Colonia,New Jersey,65,Mix Professionals Chicago,9,18,Fitness,1264,758.0
2,Kendra Valdez,1962-06-16 00:00:00,M,Press photographer,1669,2018-08-06 00:00:00,10528562,Pinckney,Michigan,61,Madison Square Park Mommies!,25,19,Parents & Family,1729,1095.0


In [42]:
# динамика количества новых членов клубов в разрезе категорий интересов с 2017 по 2020 гг.
pd.read_sql('''SELECT category_name, 
            COUNT(CASE WHEN strftime('%Y', entry_date)='2017' THEN member_name END) as count_members_2017,
            COUNT(CASE WHEN strftime('%Y', entry_date)='2018' THEN member_name END) as count_members_2018,
            COUNT(CASE WHEN strftime('%Y', entry_date)='2019' THEN member_name END) as count_members_2019,
            COUNT(CASE WHEN strftime('%Y', entry_date)='2020' THEN member_name END) as count_members_2020
            FROM members_13 GROUP BY category_name''', con)

,category_name,count_members_2017,count_members_2018,count_members_2019,count_members_2020
0,Arts & Culture,1394,1975,2047,676
1,Book Clubs,624,887,957,323
2,Cars & Motorcycles,232,344,321,122
3,Community & Environment,1337,1893,1991,625
4,Dancing,753,1162,1112,348
5,Education & Learning,1545,2107,2197,685
6,Fitness,1227,1750,1690,581
7,Food & Drink,1891,2726,2695,865
8,Games,1053,1499,1587,526
9,Health & Wellbeing,3265,4656,4727,1526
